In [1]:
cd ../dataprocessing

/Users/carverforbes/internships/google/gunshot-detection/activity-recognition/location/lbs/activity/audioset/dataprocessing


In [2]:
# import audio_processing_test as apt
import audio_processing as ap

import functools
import os
import sys

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.utils import np_utils
from sklearn import metrics
from sklearn.model_selection import train_test_split

/Users/carverforbes/.pyenv/versions/3.7.7/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
print(os.path.dirname(os.path.realpath('__file__')))

/Users/carverforbes/internships/google/gunshot-detection/activity-recognition/location/lbs/activity/audioset/dataprocessing


In [4]:
debug = False

In [5]:
src_dir = 'example_src_dir'
dest_dir = 'example_dest_dir'
# filenames should adhere to the following order
# [dataset, validation set, test set]
filenames = ['unbalanced_train_segments', 'eval_segments', 'balanced_train_segments']
labels = ['Gunshot, gunfire']
features_to_extract = ['mfcc']

In [6]:
print(src_dir)

example_src_dir


In [ ]:
length = len(filenames)
if length == 3:
    dataset_df = ap.output_df(src_dir, dest_dir, filenames[0], labels, features_to_extract)
    evaluation_df = ap.output_df(src_dir, dest_dir, filenames[1], labels, features_to_extract)
    validation_df = ap.output_df(src_dir, dest_dir, filenames[2], labels, features_to_extract)
elif length == 2:
    dataset_df = ap.output_df(src_dir, dest_dir, filenames[0], labels, features_to_extract)
    evaluation_df = ap.output_df(src_dir, dest_dir, filenames[1], labels, features_to_extract)
else:
    raise ValueError('You must have at least one dataset csv and testing data csv')

[youtube] --RFHZELSb0: Downloading webpage
[youtube] --RFHZELSb0: Downloading MPD manifest
[dashsegments] Total fragments: 23
[download] Destination: example_dest_dir/tmp/--RFHZELSb0.webm
[download] 100% of 3.01MiB in 00:107.75KiB/s ETA 00:000746
[ffmpeg] Destination: example_dest_dir/tmp/--RFHZELSb0.ogg
Deleting original file example_dest_dir/tmp/--RFHZELSb0.webm (pass -k to keep)
Download Complete
True
chopped_audio
[youtube] --hS-CvxCNQ: Downloading webpage
[download] Destination: example_dest_dir/tmp/--hS-CvxCNQ.webm
[download] 100% of 22.21MiB in 00:0284MiB/s ETA 00:002
[ffmpeg] Destination: example_dest_dir/tmp/--hS-CvxCNQ.opus
Deleting original file example_dest_dir/tmp/--hS-CvxCNQ.webm (pass -k to keep)
Download Complete
True
chopped_audio
[youtube] --hYWyUh_BY: Downloading webpage
[download] Destination: example_dest_dir/tmp/--hYWyUh_BY.webm
[download] 100% of 2.96MiB in 00:0021MiB/s ETA 00:003
[ffmpeg] Destination: example_dest_dir/tmp/--hYWyUh_BY.opus
Deleting original file 

ERROR: --h_Nif7ov4: YouTube said: This video has been removed for violating YouTube's Terms of Service.
Sorry about that.


Download Failed
[youtube] --h_ssPERp0: Downloading webpage
[download] Destination: example_dest_dir/tmp/--h_ssPERp0.webm
[download] 100% of 2.67MiB in 00:0130MiB/s ETA 00:002
[ffmpeg] Destination: example_dest_dir/tmp/--h_ssPERp0.opus
Deleting original file example_dest_dir/tmp/--h_ssPERp0.webm (pass -k to keep)
Download Complete
True
chopped_audio
[youtube] --hjjD-ANK0: Downloading webpage


ERROR: --hjjD-ANK0: YouTube said: This video has been removed for violating YouTube's Community Guidelines.
Sorry about that.


Download Failed
[youtube] --hoNP1_Usw: Downloading webpage
[download] Destination: example_dest_dir/tmp/--hoNP1_Usw.webm
[download] 100% of 2.55MiB in 00:0154MiB/s ETA 00:002
[ffmpeg] Destination: example_dest_dir/tmp/--hoNP1_Usw.opus
Deleting original file example_dest_dir/tmp/--hoNP1_Usw.webm (pass -k to keep)
Download Complete
True
chopped_audio
[youtube] --hpdFXotM0: Downloading webpage
[download] Destination: example_dest_dir/tmp/--hpdFXotM0.m4a
[download] 100% of 261.38KiB in 00:0024MiB/s ETA 00:0000
[ffmpeg] Correcting container in "example_dest_dir/tmp/--hpdFXotM0.m4a"
[ffmpeg] Post-process file example_dest_dir/tmp/--hpdFXotM0.m4a exists, skipping
Download Complete
True
chopped_audio
[youtube] --hq8wmLHHQ: Downloading webpage
[download] Destination: example_dest_dir/tmp/--hq8wmLHHQ.m4a
[download] 100% of 756.31KiB in 00:0016MiB/s ETA 00:000
[ffmpeg] Correcting container in "example_dest_dir/tmp/--hq8wmLHHQ.m4a"
[ffmpeg] Post-process file example_dest_dir/tmp/--hq8wmLHHQ.m4a ex

...

Download Failed
[youtube] -0rRNxNQSIo: Downloading webpage
[youtube] -0rRNxNQSIo: Downloading MPD manifest
[dashsegments] Total fragments: 2
[download] Destination: example_dest_dir/tmp/-0rRNxNQSIo.webm
[download] 100% of 72.56KiB in 00:019.14KiB/s ETA 00:002
[ffmpeg] Destination: example_dest_dir/tmp/-0rRNxNQSIo.ogg
Deleting original file example_dest_dir/tmp/-0rRNxNQSIo.webm (pass -k to keep)
Download Complete
True
chopped_audio
[youtube] -0rXXv6jMQ0: Downloading webpage
[youtube] -0rXXv6jMQ0: Downloading MPD manifest
[dashsegments] Total fragments: 10
[download] Destination: example_dest_dir/tmp/-0rXXv6jMQ0.webm
[download] 100% of 1.03MiB in 00:047.58KiB/s ETA 00:00:00
[ffmpeg] Destination: example_dest_dir/tmp/-0rXXv6jMQ0.opus
Deleting original file example_dest_dir/tmp/-0rXXv6jMQ0.webm (pass -k to keep)
Download Complete
True
chopped_audio
[youtube] -0rbtOMJHgQ: Downloading webpage
[download] Destination: example_dest_dir/tmp/-0rbtOMJHgQ.webm
[download] 100% of 546.72KiB in 00:008

ERROR: -0rgO-7_8x8: YouTube said: This video is unavailable.
Sorry about that.


Download Failed
[youtube] -0rk3mZ7EbU: Downloading webpage
[download] Destination: example_dest_dir/tmp/-0rk3mZ7EbU.webm
[download] 100% of 2.92MiB in 00:0044MiB/s ETA 00:003
[ffmpeg] Destination: example_dest_dir/tmp/-0rk3mZ7EbU.opus
Deleting original file example_dest_dir/tmp/-0rk3mZ7EbU.webm (pass -k to keep)
Download Complete
True
chopped_audio
[youtube] -0rs_D7KTX8: Downloading webpage
[download] Destination: example_dest_dir/tmp/-0rs_D7KTX8.webm
[download] 100% of 1.35MiB in 00:0007MiB/s ETA 00:0001
[ffmpeg] Destination: example_dest_dir/tmp/-0rs_D7KTX8.opus
Deleting original file example_dest_dir/tmp/-0rs_D7KTX8.webm (pass -k to keep)
Download Complete
True
chopped_audio
[youtube] -0ruoscQwcc: Downloading webpage
[youtube] -0ruoscQwcc: Downloading MPD manifest
[dashsegments] Total fragments: 51
[download] Destination: example_dest_dir/tmp/-0ruoscQwcc.m4a
[download] 100% of 7.51MiB in 00:454.79KiB/s ETA 00:00805
[ffmpeg] Correcting container in "example_dest_dir/tmp/-0ruoscQwcc.m

ERROR: -0sddG3eS3Y: YouTube said: This video is no longer available because the YouTube account associated with this video has been terminated.
Sorry about that.


Download Failed
[youtube] -0sfzYcIoLo: Downloading webpage
[download] Destination: example_dest_dir/tmp/-0sfzYcIoLo.webm
[download] 100% of 38.13MiB in 00:0622MiB/s ETA 00:001
[ffmpeg] Destination: example_dest_dir/tmp/-0sfzYcIoLo.opus
Deleting original file example_dest_dir/tmp/-0sfzYcIoLo.webm (pass -k to keep)
Download Complete
True
chopped_audio
[youtube] -0si9eL47HA: Downloading webpage
[download] Destination: example_dest_dir/tmp/-0si9eL47HA.webm
[download] 100% of 2.67MiB in 00:0240MiB/s ETA 00:003
[ffmpeg] Destination: example_dest_dir/tmp/-0si9eL47HA.opus
Deleting original file example_dest_dir/tmp/-0si9eL47HA.webm (pass -k to keep)
Download Complete
True
chopped_audio
[youtube] -0siyVVr9lM: Downloading webpage
[download] Destination: example_dest_dir/tmp/-0siyVVr9lM.webm
[download] 100% of 6.12MiB in 00:0164MiB/s ETA 00:006
[ffmpeg] Destination: example_dest_dir/tmp/-0siyVVr9lM.opus
Deleting original file example_dest_dir/tmp/-0siyVVr9lM.webm (pass -k to keep)
Download Comple

In [ ]:
print(datasetdf is None)

In [ ]:
datasetdf.head()

Convert features and classification labels into numpy arrays

In [ ]:
X = np.array(dataset_df.mfcc.tolist(), dtype=object)
y = np.array(dataset_df.label.tolist())

In [ ]:
def get_data_for_model(dfs):
    if len(dfs) == 3:
        return train_x, train_y, val_x, val_y, test_x, test_y
    else:
        return train_x, train_y, test_x, test_y

# Split the dataset

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 42)

In [8]:
# trying to fix bug:
# ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).
# It worked!!!
from keras import backend as K
x_train = K.cast_to_floatx(x_train)
y_train = K.cast_to_floatx(y_train)
x_test = K.cast_to_floatx(x_test)
y_test = K.cast_to_floatx(y_test)

NameError: name 'x_train' is not defined

In [ ]:
data = (x_train, x_test, y_train, y_test)

In [ ]:
def model_config_train_1(data, activation, optimizer, metrics, epochs):
    x_train, x_test, y_train, y_test = data
    model = keras.Sequential([
        keras.Input(shape=(20,)),
        keras.layers.Dense(20, activation=activation),
        keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer,
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=metrics)
    history = model.fit(x_train, y_train,
                   epochs=epochs,
                    verbose=1)
    return history

In [ ]:
def visualize_training(history, filename):
    history_dict = history.history
    history_dict.keys()
    acc = history_dict['accuracy']
    val_acc = history_dict['val_accuracy']
    loss = history_dict['loss']
    val_loss = history_dict['val_loss']

    epochs = range(1, len(acc) + 1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,7))
    
    # "bo" is for "blue dot"
    ax1.plot(epochs, loss, 'bo', label='Training loss')
    # b is for "solid blue line"
    ax1.plot(epochs, val_loss, 'b', label='Validation loss')
    ax1.set_title('Training and validation loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()
    
    ax2.plot(epochs, acc, 'bo', label='Training acc')
    ax2.plot(epochs, val_acc, 'b', label='Validation acc')
    ax2.set_title('Training and validation accuracy')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.legend(loc='lower right')
    
    plt.savefig(filename, bbox_inches='tight')
    plt.show()


In [ ]:
path = os.path.join(dest_dir, results1, '.png')
history = model_config_train_1(data, 'relu', 'adam', ['accuracy'], 20)
visualize_training(history, path)